In [1]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk import word_tokenize, pos_tag, ne_chunk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk import pos_tag
import string
swords = stopwords.words('english')
import pandas as pd
from pandas import Series,DataFrame 
%matplotlib inline 

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\hanna\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\hanna\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
# df1 = pd.read_csv('technologycompanies.csv')
# df2 = pd.read_csv('tech_1129.csv')
# df3 = pd.read_csv('hpe_and_google.csv')

In [3]:
#df.to_csv('/Users/Aifaz/Desktop/tech_all.csv')
df = pd.read_csv('tech_all.csv')

In [4]:
def get_company(url): 
    ticker = ''
    for x in url:
        if x.isupper(): 
            ticker += x
        else: 
            continue 
    return ticker 

df['ticker'] = df['web-scraper-start-url'].map(get_company)

In [5]:
def only_earningscall(title):
    title = title.lower()
    #a_list = body.split('\n')
    if 'earnings call' in title and 'transcript' in title: #a_list[0] or 'earnings call' in a_list[1]: 
        return True 
    return False
df = df[df['title'].map(only_earningscall)]

In [6]:
df['ticker'].value_counts()

GOOG     109
TMO       83
ADBE      83
AAPL      80
XRX       73
MSFT      73
AMAT      65
NCR       63
CRM       60
LDOS      59
DELL      52
AVGO      50
QCOM      49
WDC       49
BAH       48
ADI       48
SANM      45
DXC       44
NVDA      44
EBAY      44
INTC      44
NTAP      43
ORCL      42
INTU      42
CTSH      40
TXN       40
W         40
MU        40
APH       38
CSCO      37
CDW       36
IBM       36
AMD       35
HPQ       35
BKNG      33
MSI       33
JBL       33
NSIT      32
FB        29
LRCX      29
HPE       24
GOOGL      4
Name: ticker, dtype: int64

In [7]:
df[df['ticker'] == 'GOOG']['body'].iloc[0]

"Motorola Inc. (MOT) Q3 2008 Earnings Call October 30, 2008  8:00 AM ET\nExecutives\nDean Lindroth - Corporate VP of IR\nGreg Brown - Co-CEO of Motorola and CEO of Broadband Mobility Solutions\nSanjay Jha - Co-CEO of Motorola and CEO of Mobile Devices\nPaul Liska - CFO\nAnalysts\nIttai Kidron - Oppenheimer\nJim Suva - Citi\nRichard Windsor - Nomura\nMark Sue - RBC Capital Markets\nBrian Modoff - Deutsche Bank\nJeff Kvaal - Barclays Capital\nVivek Arya - Merrill Lynch\nMaynard Um - UBS\nTim Long - Banc of America Securities\nRichard Kramer - Arete Research\nEhud Gelblum - JPMorgan\nOperator\nWelcome to Motorola's third quarter 2008 earnings conference call. Today's call is being recorded. If you have any objections, please disconnect at this time. After this teleconference, the presentation material and additional financial tables will be posted on Motorola's Investor Relations website\nIn addition, a replay of this call will be available approximately three hours after the conclusion o

## Tokenizing/Lemmatizing/Stemming Documents

In [8]:
#-----------------TOKENIZING ONLY-----------------
def tokenize_body(comment):
    body = []
    comment = comment.lower()
    value = word_tokenize(comment)
    for i in value:
        if (i in swords) or (i in string.punctuation) or (i in ["\'\'", "``", "...", "\s"]):
            continue
        else: 
            body.append(i)
    return body

#-----------------TOKENIZING, LEMMATIZING, AND STEMMING -----------------
ps = PorterStemmer()
lemma = WordNetLemmatizer()
def tok_lem(text):
    body = []
    text = text.lower()
    #tokenizing and getting POS
    for w, tag in pos_tag(word_tokenize(text)):
        if (w in swords) or (w in string.punctuation) or (w in ["\'\'", "``", "...", "\s"]):
                continue
        else: 
            wntag = tag[0].lower()
            wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
            if not wntag:
                word = w
            else:
                #lemmatizing
                word = lemma.lemmatize(w, wntag)
                #stemming
                #word = ps.stem(word)
                body.append(word)
    return body

#--------------------------------------------------------------------------------
def name_lem(comment):
    pos_text = []
    for chunk in ne_chunk((pos_tag(word_tokenize(comment)))):
        if hasattr(chunk, 'label') and chunk.label() == 'PERSON':
            continue
        elif type(chunk[0]) == tuple:
            pos_text.append(chunk[0][0])
        else:
            pos_text.append(chunk[0])
            
    text = ' '.join(pos_text)
    
    body = []
    text = text.lower()      
    for w, tag in pos_tag(word_tokenize(text)):
        if (w in swords) or (w in string.punctuation) or (w in ["\'\'", "``", "...", "\s"]):
                continue
        else: 
            wntag = tag[0].lower()
            #print(w)
            #print(wntag)
            wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
            if not wntag:
                word = w
            else:
                #lemmatizing
                word = lemma.lemmatize(w, wntag)
                #print(word)
                #stemming
                #word = ps.stem(word)
                body.append(word)
    return body
    

In [217]:
#DO NOT RUN
result = df['body'].map(name_lem)

In [218]:
def join_tok(x):
    return ' '.join(x)

In [219]:
#DO NOT RUN
result = result.map(join_tok)

In [224]:
#DO NOT RUN
df['joined_tok_body'] = result

In [226]:
#df.to_csv('newdf.csv')

In [2]:
#RUN THIS INSTEAD
df = pd.read_csv('newdf.csv')
df = df.drop_duplicates(subset = 'body')
#df.to_csv('newdf2.csv')

In [43]:
df.drop([1215, 1216], inplace = True)

In [44]:
from sklearn.feature_extraction import text 
#addsw = ['quarter', 'year', 'million', 'revenue', 'growth', 'business', 'think', 'market']
#stop_words = text.ENGLISH_STOP_WORDS.union(addsw)
#If we normalize, the speakers names show up the most. We can normalize but need to account for this
vec = TfidfVectorizer(stop_words = 'english', max_df = .9)
#vec = CountVectorizer(stop_words = 'english')
X = vec.fit_transform(df['joined_tok_body'])
words = vec.get_feature_names()
dtm = pd.DataFrame(X.toarray(), columns= words)

In [45]:
trans = dtm.T
trans[0].sort_values(ascending = False)
dtm

,00,000,0021,01,0115,0122,02,03,04,05,...,zseries,zsystems,zt,ztailors,zte,ztux,zune,zurich,zynga,zyxel
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023273,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
lda = LDA(n_components=4, random_state = 2)
lda.fit(dtm)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=4, n_jobs=None,
                          perp_tol=0.1, random_state=2, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [47]:
num_words = 100
x = 1
words = vec.get_feature_names()
print('Topics')
for idx, topic in enumerate(lda.components_):
        print("\nTopic #" + str((idx+1)))
        print(" ".join([words[i] for i in topic.argsort()[-num_words+(num_words-x):-num_words:-1]]))

Topics

Topic #1
hsi theline oibda lastquarter ofthe firstquarter ppm rgus ithink merrilllynch rgu apenetration basispoints morganstanley piperjaffray aswell weare tothe thirdquarter deutschebank thomasweisel thave weadded thequarter lastquestion forthe continueto wehave needhamand robertw kindof theindustry thinkwe thiscall tomake andthe don theend themerger interms thinkyou inthe answersession thatwe grossbookings frombooking anopportunity thispoint withrespect bookingsgrowth pointsfrom littlemore weexpect ourrevenue vegot firstquestion ourguidance areconciliation ofapproximately thequestion ofattention overthe thecustomer atransaction ofoperating segmentrevenue stockrepurchases questionwould thecall wherewe comcast andamortization stronggrowth aresult supplychain thesecond thinkthat afollow thereis creativebusiness arecertainly secondquarter secondhalf increasein walterpritchard fiscal2007 toreally youmaybe thankyou thatare havetheir wetalked secondquestion thepoint businessunit cap

In [48]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda.score(X))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda.perplexity(X))

# See model parameters
#pprint(lda.get_params())

#3 topics, .9 max df
#-244687.02336053152
#11703.577592674812

#2 topics, .9 max df
#-244137.55240886947
#11459.9518518611

#5 topics, .9 max df
#-246782.66226760656
#12681.249132662582

#-245626.9261607946
#12132.381197774555

Log Likelihood:  -245118.80556521763
Perplexity:  12147.459703990215


In [49]:
import numpy as np
# Create Document - Topic Matrix
lda_output = lda.transform(X)

# column names
topicnames = ["Topic" + str(i+1) for i in range(4)]

# index names
docnames = ["Doc" + str(i+1) for i in range(len(df))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1) + 1
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.style.applymap(color_green).applymap(make_bold)
df_document_topics

In [36]:
#Doc sum of topic percentages (should = 1)
sum(lda_output[0])

0.9999999999999999

In [ ]:
df_document_topic.to_csv('doc_topics.csv')

In [307]:
df_topic_distribution = df_document_topic['dominant_topic'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,4,699
1,2,407
2,0,322
3,1,319
4,3,239


In [310]:
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [311]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda, X, vec, mds='tsne')
panel

C:\Users\hanna\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
4     -352.465363   23.442097       1        1  33.125834
2     -102.534859  317.559845       2        1  18.783291
1     -378.134094 -363.098022       3        1  18.473039
0       20.596666 -328.175171       4        1  16.416024
3      173.192337   26.292957       5        1  13.201812, topic_info=      Category           Freq        Term          Total  loglift  logprob
21888  Default   40804.000000     product   40804.000000  30.0000  30.0000
22503  Default  114164.000000     quarter  114164.000000  29.0000  29.0000
7013   Default   49875.000000    customer   49875.000000  28.0000  28.0000
539    Default    5132.000000          ad    5132.000000  27.0000  27.0000
18327  Default    4122.000000        nand    4122.000000  26.0000  26.0000
16761  Default   45856.000000      market   45856.000000  25.0000  25.0000
28757  Default   82327.000000       think   82327.000000  24.0000  24.0000
14703  Default    3578.000000      iphone    3578.000000  23.0000  23.0000
5444   Default   10049.000000       cloud   10049.000000  22.0000  22.0000
8654   Default    2936.000000        dram    2936.000000  21.0000  21.0000
22559  Default   55382.000000    question   55382.000000  20.0000  20.0000
16782  Default    4872.000000   marketing    4872.000000  19.0000  19.0000
20703  Default   12767.000000      people   12767.000000  18.0000  18.0000
4524   Default    5214.000000    capacity    5214.000000  17.0000  17.0000
14601  Default    9880.000000   inventory    9880.000000  16.0000  16.0000
30418  Default    4524.000000        user    4524.000000  15.0000  15.0000
26895  Default    4456.000000     storage    4456.000000  14.0000  14.0000
7595   Default   13217.000000      demand   13217.000000  13.0000  13.0000
7892   Default    7775.000000      device    7775.000000  12.0000  12.0000
17201  Default    2777.000000      memory    2777.000000  11.0000  11.0000
27806  Default   15719.000000  technology   15719.000000  10.0000  10.0000
16719  Default   35128.000000      margin   35128.000000   9.0000   9.0000
32133  Default   97684.000000        year   97684.000000   8.0000   8.0000
5389   Default    7861.000000      client    7861.000000   7.0000   7.0000
3473   Default   20691.000000         bit   20691.000000   6.0000   6.0000
21117  Default   10265.000000    platform   10265.000000   5.0000   5.0000
26453  Default    5299.000000    spending    5299.000000   4.0000   4.0000
25003  Default    2623.000000      search    2623.000000   3.0000   3.0000
10101  Default    8326.000000  experience    8326.000000   2.0000   2.0000
6536   Default   20480.000000        cost   20480.000000   1.0000   1.0000
...        ...            ...         ...            ...      ...      ...
25541   Topic5    1546.129065    shipment    3280.552704   1.2726  -6.1805
7013    Topic5    9273.076619    customer   49875.114799   0.3424  -4.3891
3473    Topic5    5017.936710         bit   20691.873310   0.6081  -5.0032
13988   Topic5    3239.751457    industry   10572.365694   0.8421  -5.4407
28757   Topic5   11970.779920       think   82327.738161   0.0966  -4.1338
21888   Topic5    7613.406540     product   40804.987192   0.3459  -4.5863
22559   Topic5    8912.863580    question   55382.807998   0.1980  -4.4287
32133   Topic5   12252.191021        year   97684.876568  -0.0512  -4.1105
22503   Topic5   12459.228336     quarter  114164.395440  -0.1904  -4.0938
16761   Topic5    7252.536357      market   45856.777919   0.1806  -4.6349
6536    Topic5    4525.516300        cost   20480.878647   0.5151  -5.1065
12466   Topic5    3417.490365        half   13428.157831   0.6564  -5.3873
25448   Topic5    5236.654166       share   30187.261830   0.2731  -4.9606
10049   Topic5    5838.480393      expect   37794.877461   0.1571  -4.8518
16269   Topic5    5770.384448        look   38855.037930   0.1177  -4.8635
5697    Topic5    5316.527

These two documents are both heavily weighted in the same topic, so looking to compare text

In [292]:
df.iloc[10]['body']

'SAIC, Inc. (SAI) F3Q10 Earnings Call December  8, 2008  5:00 PM ET\nExecutives\nLaura Luke – Vice President Media Relations\nWalter P. Havenstein – Chief Executive Officer & Director\nMark W. Sopp – Chief Financial Officer & Executive Vice President\nAnalysts\nJason A. Kupferberg – UBS Securities, LLC.\nRobert Spingarn – Credit Suisse\nJoseph B. Nadol, III – JPMorgan\nCai von Rumohr – Cowen & Company, LLC\nJoseph A. Vafi – Jefferies & Company, Inc.\nWilliam R. Loomis – Stifel Nicolaus & Company, Inc.\nJeff Houston – William Blair & Company\nOperator\nWelcome to the third quarter fiscal year 2010 earnings conference call. At this time all participants are in listen only mode. We will conduct a question and answer session towards the end of this conference. (Operator Instructions) I would now like to turn the presentation over to your host for today’s call Ms. Laura Luke, Vice President Media Relations.\nLaura Luke\n\nHere on today’s call are Walt Havenstein, our CEO; Mark Sopp, our CFO

In [293]:
df.iloc[40]['body']

'Xerox Corporation (NYSE:XRX) Q3 2014 Earnings Conference Call October 22, 2014 10:00 AM ET\nExecutives\n\nUrsula Burns – Chief Executive Officer\nKathy Mikells – Executive Vice President & Chief Financial Officer\nJim Firestone – Executive Vice President & President, Corporate Strategy and Asia Operations\nBob Zapfel – Executive Vice President & President, Xerox Services\n\n\n\nAnalysts\nTien-tsin Huang – JPMorgan \nShannon Cross – Cross Research \nBen Reitzes – Barclays Capital \nGeorge Tong – Piper Jaffray \nKeith Bachman – BMO Capital Markets\nAnanda Baruah – Brean Capital Markets \nBrian Essex – Morgan Stanley\nJamie Friedman – Susquehanna Financial Group\nJim Suva – Citigroup \nKulbinder Garcha – Credit Suisse \n\n\n\nOperator\nGood morning and welcome to the Xerox Corporation Q3 2014 Earnings Release Conference Call, hosted by Ursula Burns, Chairman of the Board and Chief Executive Officer. She is joined by Kathy Mikells, Executive Vice President and Chief Financial Officer.\nDu

Reference: https://www.machinelearningplus.com/nlp/topic-modeling-python-sklearn-examples/

## Sentiment Analysis

In [16]:
df

,Unnamed: 0,Unnamed: 0.1,web-scraper-order,web-scraper-start-url,title,title-href,body,ticker,joined_tok_body
0,0,0,1574739130-1421,https://seekingalpha.com/search?q=transcript&t...,eBay Q4 2006 Earnings Call Transcript,https://seekingalpha.com/article/25034-ebay-q4...,eBay Inc. (NASDAQ:EBAY) Q4 2006 Earnings Cal...,EBAY,ebay ebay q4 earnings pm transcript sponsor ex...
1,1,1,1574740471-1615,https://seekingalpha.com/search?q=transcript&t...,Salesforce.com F2Q10 (Qtr End 07/31/09) Earnin...,https://seekingalpha.com/article/157468-salesf...,Salesforce.com (NYSE:CRM) F2Q10 Earnings Call ...,CRM,salesforce.com crm august pm et executive exec...
2,2,2,1574728702-735,https://seekingalpha.com/search?q=transcript&t...,NetApp Management Discusses Q3 2013 Results - ...,https://seekingalpha.com/article/1182511-netap...,NetApp (NASDAQ:NTAP) Q3 2013 Earnings Call Feb...,NTAP,netapp ntap q3 earnings pm executive kris offi...
3,3,4,1574743420-2082,https://seekingalpha.com/search?q=transcript&t...,QUALCOMM Incorporated's CEO Discusses Q4 2011 ...,https://seekingalpha.com/article/304724-qualco...,QUALCOMM Incorporated (NASDAQ:QCOM) Q4 2011 Ea...,QCOM,qualcomm qcom q4 earnings november pm et execu...
4,4,5,1574745785-2452,https://seekingalpha.com/search?q=transcript&t...,Cisco Systems (CSCO) John T. Chambers on Q3 20...,https://seekingalpha.com/article/3181286-cisco...,"Cisco Systems, Inc. (NASDAQ:CSCO) Q3 2015 Earn...",CSCO,system inc. csco q3 earnings pm et executive v...
5,5,6,1574742760-1978,https://seekingalpha.com/search?q=transcript&t...,Broadcom's (BRCM) CEO Scott McGregor on Q2 201...,https://seekingalpha.com/article/2331665-broad...,Broadcom Corporation (BRCM) Q2 2014 Earnings C...,AVGO,corporation brcm q2 earnings conference call j...
6,6,7,1574730533-799,https://seekingalpha.com/search?q=transcript&t...,Intuit F1Q10 (Quarter End 10/31/2009) Earnings...,https://seekingalpha.com/article/174430-intuit...,Intuit Inc. (NASDAQ:INTU) F1Q10 Earnings Call ...,INTU,intuit intu november pm et executive vp financ...
7,7,8,1574742390-1918,https://seekingalpha.com/search?q=transcript&t...,Western Digital's (WDC) CEO Steve Milligan on ...,https://seekingalpha.com/article/2856706-weste...,Western Digital Corp (NASDAQ:WDC) Q2 2015 Resu...,WDC,digital wdc q2 result earnings conference call...
8,8,14,1574746367-2553,https://seekingalpha.com/search?q=transcript&t...,Electronic Data Systems Q2 2007 Earnings Call ...,https://seekingalpha.com/article/43231-electro...,Electronic Data Systems Corporation (NASDAQ:ED...,HPQ,data nasdaq ed q2 earnings august pm transcrip...
9,9,15,1574737465-1190,https://seekingalpha.com/search?q=transcript&t...,Sanmina's (SANM) CEO Bob Eulau on Q2 2018 Resu...,https://seekingalpha.com/article/4165014-sanmi...,Sanmina Corp (NASDAQ:SANM) Q2 2018 Earnings Co...,SANM,sanmina corp sanm q2 earnings conference call ...


In [11]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [12]:
def sentiment_analyzer_scores(s):
    scores = analyser.polarity_scores(s)
    #print("{:-<40} {}".format(s, str(scores)))
    #return scores
    return scores['neg'], scores['neu'], scores['pos']

In [13]:
sent = df['body'].map(sentiment_analyzer_scores)

In [14]:
neg = []
neu = []
pos = []

for i in range(len(sent)):
    print(i)
    neg.append(sent[i][0])
    neu.append(sent[i][1])
    pos.append(sent[i][2])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyError: 1151

In [ ]:
## sent_df = DataFrame()
sent_df['neg_sent'] = [x[0] for x in sent]
sent_df['neu_sent'] = [x[1] for x in sent]
sent_df['pos_sent'] = [x[2] for x in sent]

In [29]:
sent_df

,neg_sent,neu_sent,pos_sent
0,0.017,0.799,0.184
1,0.021,0.849,0.130
2,0.026,0.826,0.148
3,0.012,0.876,0.112
4,0.024,0.813,0.162
5,0.021,0.840,0.139
6,0.012,0.846,0.142
7,0.022,0.851,0.127
8,0.018,0.821,0.161
9,0.032,0.795,0.173


In [30]:
sent_df.to_csv('sentiment.csv')

## Sentiment of each word in text
Reference: https://stackoverflow.com/questions/43646877/python-extract-positive-words-from-a-string-using-sentiment-vader

In [377]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
test_subset=df['body'][0].split(' ')

sid = SentimentIntensityAnalyzer()
pos_word_list=[]
neu_word_list=[]
neg_word_list=[]

for word in test_subset:
    if (sid.polarity_scores(word)['compound']) >= 0.5:
        pos_word_list.append(word)
    elif (sid.polarity_scores(word)['compound']) <= -0.5:
        neg_word_list.append(word)
    else:
        neu_word_list.append(word)                

print('Positive :',pos_word_list)
print('\n')
print('-'*200)
#print('\n')
#print('Neutral :',neu_word_list)  
#print('\n')
#print('-'*200)
print('\n')
print('Negative :',neg_word_list) 

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\hanna\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Positive : ['best', 'free', 'FREE.', 'great', 'successes', 'strong', 'strong', 'perfectly', 'strong,', 'kind', 'richest', 'superior', 'success', 'trust', 'positive', 'amaze', 'delight', 'great', 'strong', 'success', 'excellent', 'fantastic', 'vibrant', 'enjoyed', 'success.', 'outstanding', 'sophisticated,', 'positive', 'encouraging', 'trust', 'trust', 'strong', 'excellent', 'strong', 'loyalty', 'winning', 'great', 'strong', 'free', 'great', 'trust', 'best', 'success', 'great', 'great', 'strong', 'free', 'strong', 'successful', 'strong', 'excellent', 'free', 'strong', 'strong', 'successful', 'strong', 'free', 'confidence', 'strong', 'strong', 'successes', 'strong', 'strong', 'success', 'excellent', 'impressive', 'excellent', 'strong', 'trust', 'great', 'strong.', 'free', 'confidence', 'excellent', 'impressive', 'strong', 'positive', 'free', 'strong', 'happy', 'strong', 'successful', 'strong', 'great', 'excellent', 'strong', 'successful', 'kind', 'positive', 'positive.', 'kind', 'talente

In [ ]:
returns = pd.read_csv('3d_7d_return.csv')